In [ ]:
# This script generates a sample "Monsoon Mandala" artwork using placeholder data. 
# Replace the synthetic data block with your real pandas DataFrame columns to recreate the piece with your tea farm data.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils import get_kit_measurements_df

weatherdata_df = get_kit_measurements_df(1001)

In [ ]:
weatherdata_df

In [ ]:
weatherdata_df_copy = weatherdata_df.copy()
weatherdata_df_copy.drop(columns=['kit_id', "unit","_raw"]).dropna().pivot(index="timestamp", columns='sensor', values='value')

In [ ]:
weatherdata_df.drop(columns=['kit_id', "unit","_raw"], inplace=True)
weatherdata_df.dropna(inplace=True)
weatherdata_df.index = pd.to_datetime(weatherdata_df.index)
weatherdata_df = weatherdata_df.pivot(index="timestamp", columns='sensor', values='value')
weatherdata_df.columns

In [ ]:
weatherdata_df.head(30)

In [ ]:
# ---- Mapping to polar "Monsoon Mandala" ----
# Angles map to time; radii encode a blended metric; thickness & dot size encode other variables.
df = weatherdata_df[:30]
# Interpolate as before, but add larger random noise for more "organic" variation
# Interpolate as before, then add a smooth oscillation to each column
df_interp = df.interpolate(method='time').reindex(
    pd.date_range(df.index[0], df.index[-1], periods=300)
).interpolate(method='linear')

# Add a sum of smooth oscillations (different non-harmonic frequencies) to each column
freqs = [7, 11, 15]  # example of non-harmonic frequencies
oscillation = pd.DataFrame(
    {
        col: sum(
            0.04 * np.sin(f * np.linspace(0, 2 * np.pi, len(df_interp)))
            for f in freqs
        )
        for col in df_interp.columns
    },
    index=df_interp.index
)

df = df_interp + oscillation


In [ ]:
theta = np.linspace(0, 2*np.pi, len(df), endpoint=False)

# Normalize helpers (avoid specifying colors, per instructions).
def norm(x):
    x = np.asarray(x)
    if np.nanmax(x) - np.nanmin(x) == 0:
        return np.zeros_like(x)
    return (x - np.nanmin(x)) / (np.nanmax(x) - np.nanmin(x))

T = norm(df['ftTemp'].values)
H = norm(df['gbHum'].values)
R = norm(df['NH3'].values)
W = norm(df['C3H8'].values)
L = norm(df['CO'].values)

# Radius combines temp (outer breathing), humidity (inner swell), light (diurnal bloom)
radius = 0.45 + 0.35*(0.5*T + 0.3*H + 0.2*L)

# Stroke width from wind; point size from rainfall intensity
stroke = 0.3 + 3.2*W
dots = 5 + 60*R

# Rolling medians for smooth rings
def smooth(x, k=21):
    if k < 3: 
        return x
    w = np.ones(k)/k
    return np.convolve(x, w, mode="same")

radius_smooth = smooth(radius, k=31)

# ---- Plot (no explicit colors; uses matplotlib defaults) ----
plt.figure(figsize=(8, 8))
ax = plt.subplot(111, projection="polar")
ax.set_theta_direction(-1)         # clockwise
ax.set_theta_offset(np.pi/2.0)     # start at top
ax.set_axis_off()

# Outer ribbon
ax.plot(theta, radius_smooth, linewidth=2.0)

# Inner filigree rings
for k in [3, 7, 13]:
    ax.plot(theta, smooth(radius * (0.85 + 0.05*np.sin(k*theta)), k=15), linewidth=0.8)

# Rainfall pearls
ax.scatter(theta[::3], (radius_smooth*0.92)[::3], s=dots[::3], alpha=0.6)

# Wind tick marks (radial sticks)
for th, rr, sw in zip(theta[::12], radius_smooth[::12], stroke[::12]):
    ax.plot([th, th], [rr*0.75, rr*0.98], linewidth=sw*0.12, alpha=0.8)

plt.tight_layout()
png_path = "output/monsoon_mandala_example.png"
svg_path = "output/monsoon_mandala_example.svg"
plt.savefig(png_path, dpi=300, bbox_inches="tight", pad_inches=0.05)
plt.savefig(svg_path, bbox_inches="tight", pad_inches=0.05)
png_path, svg_path

# GenAI krams

In [ ]:
from dotenv import load_dotenv
from PIL import Image
from typing import Optional
from io import BytesIO

load_dotenv()  # take environment variables from .env.

from google import genai


description = """This project uses publicly engaged artistic approaches to raise awareness of food and the environment, using a de-anthropocentric and objective approach to remediate historical narratologies and engagements for contemporary audiences.  Using microbial orientated foods as an interface to communicate scientific, cultural and political investigations into local ecologies through art, this project presents itself as a riverbank buffet of fermented foods, beverages and ideas, all intersecting through a singular themed event, along the shores of the Tamsui River. Presented as a dégustation menu, each course will consist of an edible dish, paired with a drink and an ecological artwork, taking the audience on a journey from underground, through soil and water to the sky and back with the aim of allowing participants to more clearly see the historical, present and future ecologies of chosen location for this event. 

Inspired by the Umberto Eco novel The Island of the Day Before is a project Stadon started in 2022 with the aim to explore what we can learn through revisiting historical approaches and engagement with ecology; be they Art’s overly romanticised engagement with nature and the sublime, historical artistic engagements with agriculture, food and ecology, through to more recent fetishizations of food in social media. This is also integrated with scientific research and adopts both qualitative and quantitative approaches communicating food ecologies through art and how its modes of representation and reflection have changed since Industrialisation, Urbanisation, Digitisation, Networking and Bio-Technologies, up until what we now can define as the Post-Anthropocentric Era.

Consisting of a dinner with performative, post-natural and bio-digital artworks along with a formal artist talk and an informal, conversational workshop, this event aims to ferment new perspectives on food systems, ecology, culture and society, through participation in  micro-macro sub-connectivities and entanglements in ecological systems, post-natural ideologies, microbial and fungal remedies and molecular transitions in human and non-human bodily encounters. 

This residency acted as both a research and development residency and an opportunity to try a recently developed curatorial approach of integrating, artworks, scientific findings, storytelling and performance into a multi-course themed dinner, using exclusively locally sourced foods.
"""




def pick_artstyle(description=description, model: str = "gemini-2.5-flash-image-preview") -> str:
    client = genai.Client()
    prompt = f""" '{description}'
    read this project description, and choose an artistic style to represent it with.
    Respond only with two words.

    Example output:
    Abstract Expressionism

    Cubist Geometry

    Surreal Dreamscape

    Minimal Maximalism

    Neo Baroque

    High Renaissance

    Northern Renaissance

    Pop Surrealism

    Conceptual Minimalism

    Color Field

    Geometric Abstraction

    Figurative Expressionism

    Lyrical Abstraction

    Social Realism

    Magical Realism

    Socialist Realism

    Hyper Realism

    Post Impressionism

    Neo Impressionism

    Abstract Minimalism

    Digital Collage

    Generative Aesthetics

    Vaporwave Aesthetics

    Glitch Aesthetics

    Kinetic Sculpture

    Optical Abstraction

    Constructivist Graphics

    Bauhaus Modernism

    International Style"""

    try:
        response = client.models.generate_content(
            model=model,
            contents=[prompt],
        )
    except Exception as e:
        # No credentials or API issue
        print(f"GenAI request failed: {e}")
        return None
    
    try:
        for part in response.candidates[0].content.parts:
            if getattr(part, "text", None) is not None:
                # Optional: print any textual response
                art_style = part.text
            elif getattr(part, "inline_data", None) is not None:
                output_image = Image.open(BytesIO(part.inline_data.data)).convert("RGB")
    except Exception as e:
        print(f"Failed to parse GenAI response: {e}")
        return prompt

    return art_style

In [ ]:
pick_artstyle()

